![ngo_project_image](ngo_project_image.jpg)

GoodThought NGO has been a catalyst for positive change, focusing its efforts on education, healthcare, and sustainable development to make a significant difference in communities worldwide. With this mission, GoodThought has orchestrated an array of assignments aimed at uplifting underprivileged populations and fostering long-term growth.

This project offers a hands-on opportunity to explore how data-driven insights can direct and enhance these humanitarian efforts. In this project, you'll engage with the GoodThought PostgreSQL database, which encapsulates detailed records of assignments, funding, impacts, and donor activities from 2010 to 2023. This comprehensive dataset includes:

- **`Assignments`:** Details about each project, including its name, duration (start and end dates), budget, geographical region, and the impact score.
- **`Donations`:** Records of financial contributions, linked to specific donors and assignments, highlighting how financial support is allocated and utilized.
- **`Donors`:** Information on individuals and organizations that fund GoodThought’s projects, including donor types.

Refer to the below ERD diagram for a visual representation of the relationships between these data tables:
<img src="erd.png" alt="ERD" width="50%" height="50%">


You will execute SQL queries to answer two questions, as listed in the instructions. Good luck!


In [23]:
-- highest_donation_assignments

-- CTE summarize donations amount by assignment and donor type 
WITH donations_amount AS (
	SELECT assignment_id, donor_type, ROUND(SUM(amount),2) AS rounded_total_donation_amount
	FROM donations 
	LEFT JOIN donors
	USING(donor_id)
	GROUP BY assignment_id, donor_type
)

SELECT assignment_name, region, rounded_total_donation_amount, donor_type
FROM donations_amount
LEFT JOIN assignments 
USING(assignment_id)
GROUP BY assignment_name, region, rounded_total_donation_amount, donor_type
ORDER BY rounded_total_donation_amount DESC
LIMIT 5

,assignment_name,region,rounded_total_donation_amount,donor_type
0,Assignment_3033,East,3840.66,Individual
1,Assignment_300,West,3133.98,Organization
2,Assignment_4114,North,2778.57,Organization
3,Assignment_1765,West,2626.98,Organization
4,Assignment_268,East,2488.69,Individual


In [24]:
-- top_regional_impact_assignments

-- CTE calculating number of donations for each assignment 
WITH highest_impact AS(
	SELECT assignment_id, COUNT(*) AS num_total_donations
	FROM donations
	GROUP BY assignment_id
	HAVING COUNT(*) >= 1
),
rank_assignment AS(
	SELECT assignment_name, assignment_id, region, impact_score,
	ROW_NUMBER() OVER(PARTITION BY region ORDER BY impact_score DESC) AS rank_assignment
	FROM assignments
	GROUP BY assignment_name, assignment_id, region, impact_score
)

SELECT assignment_name, region, impact_score, num_total_donations
FROM highest_impact
JOIN rank_assignment
USING(assignment_id)
WHERE rank_assignment = 1
GROUP BY region, impact_score, num_total_donations, assignment_name
ORDER BY region ASC

,assignment_name,region,impact_score,num_total_donations
0,Assignment_316,East,10.00,2
1,Assignment_2253,North,9.99,1
2,Assignment_3547,South,10.00,1
3,Assignment_2794,West,9.99,2


In [25]:
SELECT assignment_id, COUNT(*) AS num_total_donations
	FROM donations
	GROUP BY assignment_id
HAVING COUNT(*) >= 1

,assignment_id,num_total_donations
0,1489,1
1,1269,1
2,2335,3
3,4790,2
4,51,1
...,...,...
3183,1715,2
3184,55,1
3185,4827,1
3186,790,1


In [26]:
SELECT assignment_name, assignment_id, region, impact_score,
	ROW_NUMBER() OVER(PARTITION BY region ORDER BY impact_score DESC) AS rank_assignment
	FROM assignments
	GROUP BY assignment_name, assignment_id, region, impact_score

,assignment_name,assignment_id,region,impact_score,rank_assignment
0,Assignment_316,316,East,10.00,1
1,Assignment_1514,1514,East,9.99,2
2,Assignment_3624,3624,East,9.98,3
3,Assignment_3196,3196,East,9.97,4
4,Assignment_2281,2281,East,9.95,5
...,...,...,...,...,...
4995,Assignment_3778,3778,West,1.05,1250
4996,Assignment_1150,1150,West,1.05,1251
4997,Assignment_2518,2518,West,1.04,1252
4998,Assignment_1033,1033,West,1.02,1253
